# Lab 2 Assignment

__Name:__ Priyanka Manoj Shewale

In the assignment, you will analyze which factors are associated with the number of likes a Facebook post gets. The dataset is from https://archive.ics.uci.edu/ml/datasets/Facebook+metrics. All posts are from the page of a renowned cosmetics brand.

The dependent variable is `like`.

In [80]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [81]:
fb_like = pd.read_csv('http://fengmai.net/download/data/bia652/dataset_Facebook.csv', sep=";")
# remove unneeded variables
not_needed = ['Page total likes', 'Total Interactions']
fb_like = fb_like.drop(not_needed, axis = 1)

In [82]:
fb_like.head(5)

,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share
0,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0
1,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0
2,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0
3,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0
4,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0


## Q1 - Modeling with categorical Xs

- Use [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to create dummy variables for the following variables: `Type`, `Category`, `Post Month`, `Post Weekday`, `Post Hour`. Make sure that `drop_firt=True` to avoid multicollinearity problem.
- You may want to read [this post](https://stackoverflow.com/questions/24109779/running-get-dummies-on-several-dataframe-columns) to create dummies for multiple columns.
- The origninal columns will be automatically dropped after you create the dummies. 
- You may want to check if any rows have missing variables and drop them accordingly (use [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)). 
- Create a regression model that uses all the variables (other than like) as independent variables. You'll need to use `statsmodels` and not `sklearn`. Display the OLS Regression Results.  
- No need to split the data to train-test sets for this exercise. 

In [83]:
#### YOUR CODE HERE ####
dfo=pd.get_dummies(data=fb_like, columns=['Type', 'Category','Post Month','Post Weekday','Post Hour'],drop_first=True)
df=dfo.dropna(axis=0,how='any')
x_variable=df.drop('like',axis=1)
x_variable=sm.add_constant(x_variable)
y_variable=df.like
model = sm.OLS(y_variable, x_variable)
result=model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   like   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     937.5
Date:                Thu, 07 Apr 2022   Prob (F-statistic):               0.00
Time:                        02:53:11   Log-Likelihood:                -2387.3
No. Observations:                 495   AIC:                             4885.
Df Residuals:                     440   BIC:                             5116.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Q2 Bootstrap confidence interval

Your tasks is to use the boostrap method to construct a 95% CI for the coefficients for **comment** and **share** using the same model as in Q1. Do you get similar CI to Q1? Why or why not?

In [84]:
! pip install mlxtend
! pip install -U statsmodels
import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
from itertools import cycle
from tqdm.auto import tqdm
from scipy import interpolate
from sklearn import preprocessing
from sklearn import datasets

In [85]:
bootstrapped_ests = []
for i in tqdm(range(5000)):
    # YOUR CODE HERE 
    fb_like_boot = df.sample(n=len(df), replace=True)
    fb_model_boot = smf.ols(formula='like~comment+share', data=fb_like_boot).fit()
    bootstrapped_ests.append(fb_model_boot.params[['comment','share']])
    
b_comments, b_shares = zip(*bootstrapped_ests)

print("The 95% bootstrapped CI of b_comments is [{:.2f}, {:.2f}].".format(
    np.percentile(b_comments, 2.5),
    np.percentile(b_comments, 97.5)))

print("The 95% bootstrapped CI of b_shares is [{:.2f}, {:.2f}].".format(
    np.percentile(b_shares, 2.5),
    np.percentile(b_shares, 97.5)))

  0%|          | 0/5000 [00:00<?, ?it/s]

The 95% bootstrapped CI of b_comments is [2.07, 5.33].
The 95% bootstrapped CI of b_shares is [4.70, 7.04].


## Q3 - Use AIC/BIC for model selection

- Read the [example here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py) and the [manual here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsIC.html#sklearn.linear_model.LassoLarsIC). You may want to create a separate notebook to run the example.
- Use `LassoLarsIC` to select the best model using the **`bic`** criterion. 
- There is no need to split the data to train-test sets for this exercise. 
- Which X variables are selected by the `LassoLarsIC` method? Answer using variable names. 

In [86]:
#### YOUR CODE HERE ####
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import linear_model

In [87]:
#LassoLarsIc model using bic 
model_bic = linear_model.LassoLarsIC(criterion='bic')
model_bic.fit(x_variable, y_variable)
model_bic.coef_
#alpha_bic_ = model_bic.alpha_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:138: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,


array([ 0.00000000e+00,  7.39631746e-01, -1.25444860e-04, -1.58522447e-05,
        9.11242490e-01, -9.12740618e-01,  3.80946209e-03,  0.00000000e+00,
        1.29521037e-03,  3.46663490e-02,  9.64880172e-01,  1.38857964e+00,
        0.00000000e+00, -2.93752069e+01,  0.00000000e+00,  8.66328204e+00,
        1.10202659e+01,  1.81045603e+00,  4.55791604e+00,  1.31989681e+00,
        7.18636470e+00, -2.98511964e+00,  0.00000000e+00,  5.61162022e+00,
        8.59759110e+00,  1.17206876e+01, -5.31766745e+00, -1.04309016e+01,
       -1.94374071e+00,  0.00000000e+00,  3.40630897e+00, -1.71314293e+00,
       -2.55488286e+00, -3.74662285e+00,  0.00000000e+00,  0.00000000e+00,
        6.73991606e-01, -6.27725532e+00,  1.21477912e+01,  0.00000000e+00,
        0.00000000e+00, -2.88672223e+00, -2.85472318e+00,  0.00000000e+00,
       -5.86870756e+00,  0.00000000e+00,  1.03424431e-01,  0.00000000e+00,
       -2.97120950e+00,  3.36168263e+01,  2.62372032e-01,  0.00000000e+00,
        2.46979061e+01,  

In [88]:
x_variable.columns[np.abs(model_bic.coef_) > 0] #X variables are selected by the LassoLarsIC method

Index(['Paid', 'Lifetime Post Total Reach', 'Lifetime Post Total Impressions',
       'Lifetime Engaged Users', 'Lifetime Post Consumers',
       'Lifetime Post Consumptions',
       'Lifetime Post reach by people who like your Page',
       'Lifetime People who have liked your Page and engaged with your post',
       'comment', 'share', 'Type_Status', 'Category_2', 'Category_3',
       'Post Month_2', 'Post Month_3', 'Post Month_4', 'Post Month_5',
       'Post Month_6', 'Post Month_8', 'Post Month_9', 'Post Month_10',
       'Post Month_11', 'Post Month_12', 'Post Weekday_2', 'Post Weekday_4',
       'Post Weekday_5', 'Post Weekday_6', 'Post Weekday_7', 'Post Hour_4',
       'Post Hour_5', 'Post Hour_6', 'Post Hour_9', 'Post Hour_10',
       'Post Hour_12', 'Post Hour_14', 'Post Hour_16', 'Post Hour_17',
       'Post Hour_18', 'Post Hour_20'],
      dtype='object')

## Q4 - Compare out-of-sample perormance

- Split the data to 90% train and 10% test set. 
- Train three models using the train set:
   * `LassoLarsIC` model (a model that uses BIC to select variables)
   * a full model (a model that includes all X variables)
   * a `LassoCV` model (a model that uses CV on train set to select variables)
- Remember to train a scaler using train set only, and transform both X_train and X_test. 
- Compare their test-set performance using MSE. Which model has the best performance? 

In [89]:
#### YOUR CODE HERE ####
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LassoCV, lars_path
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(x_variable, y_variable, test_size=0.1)
scaler = preprocessing.StandardScaler().fit(X_train)

LassoCV Model

In [90]:
model_lassoCV = LassoCV(cv=5, normalize=False) #don't need to normalize again
model_lassoCV.fit(scaler.transform(X_train), y_train) # transform X_train

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


LassoCV(cv=5, normalize=False)

In [91]:
y_hat_test = model_lassoCV.predict(scaler.transform((X_test)))
metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_test)

3272.9540482321663

In [92]:
model_lassoLarsIC = linear_model.LassoLarsIC(criterion='bic') #don't need to normalize again
model_lassoLarsIC.fit(scaler.transform(X_train), y_train) # transform X_train

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:138: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,


LassoLarsIC(criterion='bic')

LassoIC Model

In [93]:
y_hat_test1 = model_lassoLarsIC.predict(scaler.transform((X_test))) 
metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_test1)

3275.105377521776

Full Model

In [94]:
ols1=LinearRegression()
ols1.fit(scaler.transform(X_train), y_train)

LinearRegression()

In [95]:
yp=ols1.predict(scaler.transform((X_test))) 

In [96]:
metrics.mean_squared_error(y_true=y_test, y_pred=yp)

3448.22492820034

Solution : Observe MSE Values, LassoCV is the best model